In [1]:
%load_ext autoreload
%autoreload 2

import os
import glob
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import dask.dataframe as dd

# 1. Import Dataset

In [2]:
data_folder = str(os.getcwd()) + "\data"
orig_filename = "/sample_orig_*.txt"


orig_headers = ['CREDIT_SCORE','FIRST_PAYMENT_DATE','FIRST_TIME_HOMEBUYER_FLAG','MATURITY_DATE','MSA','MI_PCT',
                'NUMBER_OF_UNITS','OCCUPANCY_STATUS','CLTV','DTI','ORIGINAL_UPB','LTV','ORIGINAL_INTEREST_RATE',
                'CHANNEL','PPM','AMORTIZATION_TYPE','PROPERTY_STATE', 'PROPERTY_TYPE','POSTAL_CODE',
                'LOAN_SEQUENCE_NUMBER','LOAN_PURPOSE', 'ORIGINAL_LOAN_TERM','NUMBER_OF_BORROWERS','SELLER_NAME',
                'SERVICER_NAME','SUPER_CONFORMING_FLAG','PRE-RELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER', 
                'PROGRAM_INDICATOR', 'RELIEF_REFINANCE_INDICATOR', 'PROPERTY_VALUATION_METHOD', 'IO_INDICATOR']

raw_orig_types = {'CREDIT_SCORE':float, 'FIRST_PAYMENT_DATE':str, 'FIRST_TIME_HOMEBUYER_FLAG':str, 'MATURITY_DATE':str,
              'MSA':str, 'MI_PCT':float, 'NUMBER_OF_UNITS':float, 'OCCUPANCY_STATUS':str, 'CLTV':float, 'DTI':float,
              'ORIGINAL_UPB':float, 'LTV':float, 'ORIGINAL_INTEREST_RATE':float, 'CHANNEL':str, 'PPM': str,
              'AMORTIZATION_TYPE':str ,'PROPERTY_STATE':str, 'PROPERTY_TYPE':str ,'POSTAL_CODE':str,
              'LOAN_SEQUENCE_NUMBER':str, 'LOAN_PURPOSE':str, 'ORIGINAL_LOAN_TERM':float, 'NUMBER_OF_BORROWERS':float,
              'SELLER_NAME': str, 'SERVICER_NAME':str, 'SUPER_CONFORMING_FLAG':str,
              'PRE-RELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER':str, 'PROGRAM_INDICATOR':str, 'RELIEF_REFINANCE_INDICATOR':str,
              'PROPERTY_VALUATION_METHOD':str, 'IO_INDICATOR':str}

mutated_orig_types = raw_orig_types.copy()

orig_dates = ['FIRST_PAYMENT_DATE', 'MATURITY_DATE']


perf_filename = "\sample_svcg_*.txt"


perf_headers = ['LOAN_SEQUENCE_NUMBER','MONTHLY_REPORTING_PERIOD','CURRENT_ACTUAL_UPB',
                'CURRENT_LOAN_DELINQUENCY_STATUS','LOAN_AGE','REMAINING_MONTHS_TO_LEGAL_MATURITY', 
                'DEFECT_SETTLEMENT_DATE','MODIFICATION_FLAG', 'ZERO_BALANCE_CODE', 
                'ZERO_BALANCE_EFFECTIVE_DATE','CURRENT_INTEREST_RATE','CURRENT_DEFERRED_UPB','DDLPI',
                'MI_RECOVERIES', 'NET_SALE_PROCEEDS','NON_MI_RECOVERIES','EXPENSES', 'LEGAL_COSTS',
                'MAINTENANCE_AND_PRESERVATION_COSTS','TAXES_AND_INSURANCE','MISCELLANEOUS_EXPENSES',
                'ACTUAL_LOSS_CALCULATION', 'MODIFICATION_COST','STEP_MODIFICATION_FLAG','DEFERRED_PAYMENT_PLAN',
                'ELTV','ZERO_BALANCE_REMOVAL_UPB','DELINQUENT_ACCRUED_INTEREST','DELINQUENCY_DUE_TO_DISASTER',
                'BORROWER_ASSISTANCE_STATUS_CODE','CURRENT_MONTH_MODIFICATION_COST','INTEREST_BEARING_UPB']

raw_perf_types = {'LOAN_SEQUENCE_NUMBER':str, 'MONTHLY_REPORTING_PERIOD':str, 'CURRENT_ACTUAL_UPB':float,
              'CURRENT_LOAN_DELINQUENCY_STATUS': str, 'LOAN_AGE':float, 'REMAINING_MONTHS_TO_LEGAL_MATURITY':float, 
              'DEFECT_SETTLEMENT_DATE':str, 'MODIFICATION_FLAG':str, 'ZERO_BALANCE_CODE':str,
              'ZERO_BALANCE_EFFECTIVE_DATE':str, 'CURRENT_INTEREST_RATE':float, 'CURRENT_DEFERRED_UPB':float,
              'DDLPI': str,'MI_RECOVERIES':float, 'NET_SALE_PROCEEDS':str ,'NON_MI_RECOVERIES':float,'EXPENSES':float,
              'LEGAL_COSTS':float ,'MAINTENANCE_AND_PRESERVATION_COSTS':float, 'TAXES_AND_INSURANCE':float,
              'MISCELLANEOUS_EXPENSES':float,'ACTUAL_LOSS_CALCULATION':float, 'MODIFICATION_COST':float,
              'STEP_MODIFICATION_FLAG':str, 'DEFERRED_PAYMENT_PLAN':str, 'ELTV':float, 'ZERO_BALANCE_REMOVAL_UPB':float,
              'DELINQUENT_ACCRUED_INTEREST':float, 'DELINQUENCY_DUE_TO_DISASTER':str,'BORROWER_ASSISTANCE_STATUS_CODE':str,
              'CURRENT_MONTH_MODIFICATION_COST':float, 'INTEREST_BEARING_UPB':float}

mutated_perf_types = raw_perf_types.copy()
mutated_perf_types['CURRENT_LOAN_DELINQUENCY_STATUS'] = float

perf_dates = ['DEFECT_SETTLEMENT_DATE', 'ZERO_BALANCE_EFFECTIVE_DATE', 'DDLPI']

In [ ]:
from FREDDIEMAC_offline_preprocessing import pipeline_from_raw_data

pipeline_from_raw_data(orig_filename, 
                        perf_filename,
                        orig_headers,
                        perf_headers,
                        raw_orig_types, 
                        raw_perf_types,
                        mutated_orig_types,
                        mutated_perf_types,
                        data_folder)

In [ ]:
df_orig = dd.read_parquet(data_folder + "/sample_orig_*.parquet.gzip")
df_perf = dd.read_parquet(data_folder + "/sample_svcg_*.parquet.gzip")

In [ ]:
print(df_perf["LOAN_SEQUENCE_NUMBER"].nunique().compute())
print(df_orig["LOAN_SEQUENCE_NUMBER"].nunique().compute())

In [ ]:
df_orig["ORIGINAL_LOAN_TERM"].value_counts().compute()

In [51]:
from FREDDIEMAC_classification import assign_labels_to_orig, assign_labels_to_perf, cutoff_sequence_according_to_label
from FREDDIEMAC_reducing import drop_short_sequences, drop_long_sequences, select_specific_original_loan_term

offline_preprocessed_orig_filename = "/sample_orig_*.parquet.gzip"
offline_preprocessed_perf_filename = "/sample_svcg_*.parquet.gzip"

annual_dataset_paths_orig = glob.glob(data_folder + offline_preprocessed_orig_filename)
annual_dataset_paths_perf = glob.glob(data_folder + offline_preprocessed_perf_filename)

annual_dataset_iterator = tqdm(zip(annual_dataset_paths_orig, annual_dataset_paths_perf))

for path_orig, path_perf  in annual_dataset_iterator:
    annual_dataset_iterator.set_description("Working on %s and %s" % (Path(path_orig).stem, Path(path_perf).stem))

    annual_df_orig = pd.read_parquet(path_orig)
    annual_df_orig = annual_df_orig.astype(mutated_orig_types)

    annual_df_perf = pd.read_parquet(path_perf)
    annual_df_perf = annual_df_perf.astype(mutated_perf_types)

    annual_df_orig, annual_df_perf = select_specific_original_loan_term(annual_df_orig, annual_df_perf, loan_terms_list=[180])

    annual_df_orig = assign_labels_to_orig(annual_df_orig, annual_df_perf)
    annual_df_perf = assign_labels_to_perf(annual_df_orig, annual_df_perf)
    annual_df_orig, annual_df_perf = cutoff_sequence_according_to_label(annual_df_orig, annual_df_perf)
    annual_df_orig, annual_df_perf = drop_short_sequences(annual_df_orig, annual_df_perf, 12)
    annual_df_orig, annual_df_perf = drop_long_sequences(annual_df_orig, annual_df_perf, 180)
    #TODO annual_df_orig, annual_df_perf = drop_modified_loans(annual_df_orig, annual_df_perf)

    annual_df_orig.to_parquet(Path(data_folder) / ("labeled_" + str(Path(path_orig).stem) + ".gzip"), compression="gzip")
    annual_df_perf.to_parquet(Path(data_folder) / ("labeled_" + str(Path(path_perf).stem) + ".gzip"), compression="gzip")


Working on sample_orig_2006.parquet and sample_svcg_2006.parquet: : 8it [10:52, 81.53s/it] 


### 1.1 Data Exploration

In [60]:
df_orig = dd.read_parquet(data_folder + "/labeled_sample_orig_*.parquet.gzip")
df_perf = dd.read_parquet(data_folder + "/labeled_sample_svcg_*.parquet.gzip")

In [61]:
print(df_perf["LOAN_SEQUENCE_NUMBER"].nunique().compute())
print(df_orig["LOAN_SEQUENCE_NUMBER"].nunique().compute())

86654
86659


In [62]:
df_orig["ORIGINAL_LOAN_TERM"].value_counts().compute()

180.0    86659
Name: ORIGINAL_LOAN_TERM, dtype: int64

In [63]:
df_orig["LABEL"].value_counts().compute()

0    82989
2     2740
1      908
3       22
Name: LABEL, dtype: int64

In [64]:
print(df_orig["TOTAL_OBSERVED_LENGTH"].min().compute())
print(df_orig["TOTAL_OBSERVED_LENGTH"].max().compute())

12.0
180.0


In [65]:
a = df_orig["TOTAL_OBSERVED_LENGTH"].value_counts().compute()
a[a < 180].sort_values(ascending=False)

Series([], Name: TOTAL_OBSERVED_LENGTH, dtype: int64)

In [66]:
df_perf[df_perf["TOTAL_OBSERVED_LENGTH"] > 180]["LOAN_SEQUENCE_NUMBER"].unique().head(30)

Series([], Name: LOAN_SEQUENCE_NUMBER, dtype: object)

In [67]:
df_perf[df_perf["MODIFICATION_FLAG"] == "Y"].compute().count()

LOAN_SEQUENCE_NUMBER                  231
MONTHLY_REPORTING_PERIOD              231
CURRENT_ACTUAL_UPB                    231
CURRENT_LOAN_DELINQUENCY_STATUS       231
LOAN_AGE                              231
REMAINING_MONTHS_TO_LEGAL_MATURITY    231
DEFECT_SETTLEMENT_DATE                231
MODIFICATION_FLAG                     231
ZERO_BALANCE_CODE                     231
ZERO_BALANCE_EFFECTIVE_DATE           231
CURRENT_INTEREST_RATE                 231
CURRENT_DEFERRED_UPB                  231
DDLPI                                 231
MI_RECOVERIES                         231
NET_SALE_PROCEEDS                     231
NON_MI_RECOVERIES                     231
EXPENSES                              231
LEGAL_COSTS                           231
MAINTENANCE_AND_PRESERVATION_COSTS    231
TAXES_AND_INSURANCE                   231
MISCELLANEOUS_EXPENSES                231
ACTUAL_LOSS_CALCULATION               231
MODIFICATION_COST                     231
STEP_MODIFICATION_FLAG            

# 2. Aggregating data like Blumenstock et al. 2020 experiment 4.1

In [68]:
from FREDDIEMAC_reducing import reduce_length_of_sequence
from FREDDIEMAC_aggregation import aggregate_to_blumenstock_exp4

labeled_orig_filename = "/labeled_sample_orig_*.parquet.gzip"
labeled_perf_filename = "/labeled_sample_svcg_*.parquet.gzip"

annual_dataset_paths_orig = glob.glob(data_folder + labeled_orig_filename)
annual_dataset_paths_perf = glob.glob(data_folder + labeled_perf_filename)

annual_dataset_iterator = tqdm(zip(annual_dataset_paths_orig, annual_dataset_paths_perf))

for path_orig, path_perf  in annual_dataset_iterator:
    annual_dataset_iterator.set_description("Working on %s and %s" % (Path(path_orig).stem, Path(path_perf).stem))

    annual_df_orig = pd.read_parquet(path_orig)
    annual_df_orig = annual_df_orig.astype(mutated_orig_types)

    annual_df_perf = pd.read_parquet(path_perf)
    annual_df_perf = annual_df_perf.astype(mutated_perf_types)

    shortened_annual_df_perf = reduce_length_of_sequence(annual_df_perf, length=12)
    df_blumenstock = aggregate_to_blumenstock_exp4(annual_df_orig, shortened_annual_df_perf)

    df_blumenstock.to_parquet(Path(data_folder) / ("blumenstock_" + str(Path(path_orig).stem) + ".gzip"), compression="gzip")

Working on labeled_sample_orig_2006.parquet and labeled_sample_svcg_2006.parquet: : 8it [04:56, 37.04s/it]


In [69]:
blumenstock_types = {'LOAN_SEQUENCE_NUMBER': str, 'INT_RATE': float, 'ORIG_UPB': float, 'FICO_SCORE': float,
                    'DTI_R': float, 'LTV_R': float, 'FIRST_PAYMENT_DATE': str, 'BAL_REPAID': float, 'T_ACT_12M': float, 'T_DEL_30D': float, 
                    'T_DEL_60D': float, 'LABEL': str, 'REMAINING_MONTHS_TO_LEGAL_MATURITY': float, "TIME_TO_EVENT": float, 'TOTAL_OBSERVED_LENGTH': float}


df_blumenstock = dd.read_parquet(data_folder + "/blumenstock_labeled_sample_orig_*.parquet.gzip")
df_blumenstock = df_blumenstock.astype(blumenstock_types)

In [70]:
df_blumenstock.head(20)

,LOAN_SEQUENCE_NUMBER,INT_RATE,ORIG_UPB,FICO_SCORE,DTI_R,LTV_R,FIRST_PAYMENT_DATE,REMAINING_MONTHS_TO_LEGAL_MATURITY,TOTAL_OBSERVED_LENGTH,TIME_TO_EVENT,LABEL,BAL_REPAID,T_ACT_12M,T_DEL_30D,T_DEL_60D
0,F99Q10000064,5.250,107000.0,619.0,25.0,74.0,200303,56.0,125.0,125.0,0,0.389586,12.0,0.0,0.0
1,F99Q10000252,6.375,190000.0,702.0,57.0,70.0,199903,147.0,33.0,33.0,0,0.880235,12.0,0.0,0.0
2,F99Q10000601,7.250,53000.0,660.0,39.0,80.0,199904,154.0,25.0,25.0,1,0.913498,11.0,1.0,0.0
3,F99Q10000863,7.000,47000.0,765.0,27.0,50.0,199905,149.0,32.0,32.0,0,0.854563,12.0,0.0,0.0
4,F99Q10001212,6.125,96000.0,643.0,20.0,59.0,199905,2.0,158.0,158.0,2,0.016883,12.0,0.0,0.0
5,F99Q10001222,6.000,180000.0,683.0,18.0,60.0,199905,128.0,32.0,32.0,0,0.796372,12.0,0.0,0.0
6,F99Q10001366,6.250,53000.0,728.0,12.0,55.0,199905,80.0,78.0,78.0,0,0.553535,12.0,0.0,0.0
7,F99Q10001375,6.250,240000.0,782.0,33.0,47.0,199905,107.0,51.0,51.0,0,0.693716,12.0,0.0,0.0
8,F99Q10001577,6.500,61000.0,760.0,999.0,65.0,199905,138.0,16.0,16.0,0,0.845096,12.0,0.0,0.0
9,F99Q10001666,6.500,85000.0,804.0,999.0,59.0,199903,138.0,14.0,14.0,0,0.811232,12.0,0.0,0.0


# 3. Expanding Blumenstock et al. 2020 experiment 4.1 to dynamic data

In [71]:
from FREDDIEMAC_aggregation import aggregate_to_blumenstock_exp4_dynamic

labeled_orig_filename = "/labeled_sample_orig_*.parquet.gzip"
labeled_perf_filename = "/labeled_sample_svcg_*.parquet.gzip"

annual_dataset_paths_orig = glob.glob(data_folder + labeled_orig_filename)
annual_dataset_paths_perf = glob.glob(data_folder + labeled_perf_filename)

annual_dataset_iterator = tqdm(zip(annual_dataset_paths_orig, annual_dataset_paths_perf))

for path_orig, path_perf  in annual_dataset_iterator:
    annual_dataset_iterator.set_description("Working on %s and %s" % (Path(path_orig).stem, Path(path_perf).stem))

    annual_df_orig = pd.read_parquet(path_orig)
    annual_df_orig = annual_df_orig.astype(mutated_orig_types)

    annual_df_perf = pd.read_parquet(path_perf)
    annual_df_perf = annual_df_perf.astype(mutated_perf_types)

    df_blumenstock_dynamic = aggregate_to_blumenstock_exp4_dynamic(annual_df_orig, annual_df_perf)

    df_blumenstock_dynamic.to_parquet(Path(data_folder) / ("blumenstock_dynamic_" + str(Path(path_orig).stem) + ".gzip"), compression="gzip")

Working on labeled_sample_orig_2006.parquet and labeled_sample_svcg_2006.parquet: : 8it [00:46,  5.83s/it]


In [72]:
blumenstock_dynamic_types = {'LOAN_SEQUENCE_NUMBER': str, 'MONTHLY_REPORTING_PERIOD': str,'CURRENT_ACTUAL_UPB': float, 'CURRENT_LOAN_DELINQUENCY_STATUS': float, 
                    'CURRENT_INTEREST_RATE':float,'ELTV': float ,'LOAN_AGE': float, 'REMAINING_MONTHS_TO_LEGAL_MATURITY': float, 'CREDIT_SCORE': float,
                    'DTI': float, 'LTV': float, 'BAL_REPAID': float, 
                    'LABEL': str, "TIME_TO_EVENT": float, 'ORIGINAL_INTEREST_RATE': float, 'ORIGINAL_UPB': float, 'TOTAL_OBSERVED_LENGTH': float}

df_blumenstock_dynamic = dd.read_parquet(data_folder + "/blumenstock_dynamic_labeled_sample_orig_*.parquet.gzip")
df_blumenstock_dynamic = df_blumenstock_dynamic.astype(blumenstock_dynamic_types)

In [73]:
df_blumenstock_dynamic.head(20)

,CURRENT_INTEREST_RATE,ELTV,CURRENT_ACTUAL_UPB,CURRENT_LOAN_DELINQUENCY_STATUS,MONTHLY_REPORTING_PERIOD,REMAINING_MONTHS_TO_LEGAL_MATURITY,LOAN_AGE,TOTAL_OBSERVED_LENGTH,TIME_TO_EVENT,LABEL,LOAN_SEQUENCE_NUMBER,ORIGINAL_INTEREST_RATE,ORIGINAL_UPB,CREDIT_SCORE,DTI,LTV,BAL_REPAID
0,5.25,0.0,107000.00,0.0,200302,180.0,0.0,125.0,125.0,0,F99Q10000064,5.25,107000.0,619.0,25.0,74.0,1.000000
1,5.25,0.0,106000.00,0.0,200303,179.0,1.0,125.0,125.0,0,F99Q10000064,5.25,107000.0,619.0,25.0,74.0,0.990654
2,5.25,0.0,106000.00,0.0,200304,178.0,2.0,125.0,125.0,0,F99Q10000064,5.25,107000.0,619.0,25.0,74.0,0.990654
3,5.25,0.0,106000.00,0.0,200305,177.0,3.0,125.0,125.0,0,F99Q10000064,5.25,107000.0,619.0,25.0,74.0,0.990654
4,5.25,0.0,106000.00,0.0,200306,176.0,4.0,125.0,125.0,0,F99Q10000064,5.25,107000.0,619.0,25.0,74.0,0.990654
5,5.25,0.0,105000.00,0.0,200307,175.0,5.0,125.0,125.0,0,F99Q10000064,5.25,107000.0,619.0,25.0,74.0,0.981308
6,5.25,0.0,105000.00,0.0,200308,174.0,6.0,125.0,125.0,0,F99Q10000064,5.25,107000.0,619.0,25.0,74.0,0.981308
7,5.25,0.0,104394.26,0.0,200309,173.0,7.0,125.0,125.0,0,F99Q10000064,5.25,107000.0,619.0,25.0,74.0,0.975647
8,5.25,0.0,103526.73,0.0,200310,172.0,8.0,125.0,125.0,0,F99Q10000064,5.25,107000.0,619.0,25.0,74.0,0.967540
9,5.25,0.0,103116.97,0.0,200311,171.0,9.0,125.0,125.0,0,F99Q10000064,5.25,107000.0,619.0,25.0,74.0,0.963710
